In [ ]:
from avcv.all import *

In [ ]:
cc = DiagnoseCoco('/data/DMS_Behavior_Detection/Action_Eating_frames/annotations/instances_default.json')

In [ ]:
def get_video_name(img):
    vname = img['file_name'].split('/')[-2]
    return vname[:-12], vname
vname, vname_split = get_video_name(img)

In [ ]:
video_imgs = dict()
for img_id in cc.img_ids:
    img = cc.gt.imgs[img_id]
    vname = get_video_name(img)
    if not vname in video_imgs: video_imgs[vname] = []
    video_imgs[vname].append(img_id)

# for vname in list(sorted(video_imgs, key=lambda x:x[1])):
#     print(vname)

In [ ]:
def vname2_path(vname):
    return f'/data/DMS_Behavior_Detection/RawVideos/Action_Eating/{vname[0]}/{vname[1]}.mp4'

In [ ]:
vnames = list(video_imgs.keys())

In [ ]:
vname = vnames[0]

In [ ]:
video = mmcv.VideoReader(vname2_path(vname))

In [ ]:
vlen = len(video)
nframes = len(list(sorted(video_imgs[vname])))
sampling_rate = vlen/nframes

In [ ]:
def ann2bbox(ann):
    x,y,w,h = ann['bbox']
    return np.array([x,y,x+w,y+h])

In [ ]:
class_names = ['None']+[cat['name'] for cat in cc.gt.cats.values()]

In [ ]:
vis = []
# video_anns = dict()
video_index_to_anns = dict()

for i, img_id in tqdm(enumerate(list(sorted(video_imgs[vname])))):
    anns = cc.gt.imgToAnns[img_id]
    img = cc.gt.imgs[img_id]
    bboxes = np.array([ann2bbox(ann) for ann in anns])
    cats = [ann['category_id'] for ann in anns]
    scores = [1]*len(bboxes)

    # v = bbox_visualize(cc.imread(img_id), bboxes, scores, cats, class_names=class_names)
    vis.append(v)
    
    # video_anns[i] = len(cats)
    video_index_to_anns[i] = (bboxes, scores, cats)

79it [00:00, 46926.78it/s]


In [ ]:
vis = []
for i, frame in tqdm(enumerate(video), total=len(video)):
    index = int(i/sampling_rate)
    (bboxes, scores, cats) = video_index_to_anns[index]
    v = bbox_visualize(frame, bboxes, scores, cats, class_names=class_names)
    vis.append(v)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 472/472 [00:02<00:00, 206.00it/s]


In [ ]:
images_to_video(vis, 'vis.mp4', output_size=(600, 400))

2022-09-18 10:24:16.984 | INFO     | avcv.utils:images_to_video:267 - Write video, output_size: (600, 400)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 472/472, 84.2 task/s, elapsed: 6s, ETA:     0s

2022-09-18 10:24:22.592 | INFO     | avcv.utils:images_to_video:277 - -> /home/anhvth8/gitprojects/YOLOX/nbs/vis.mp4


In [ ]:
# bboxes.shape

In [ ]:
# img